## Objective 1 – Data Handling & Processing

**1. Data Cleaning & Preprocessing**

In [ ]:
import pandas as pd
df1 = pd.read_csv(r"C:\Users\samiksha.Godghate\Desktop\Assignment\df_2021.csv")
df2 = pd.read_csv(r"C:\Users\samiksha.Godghate\Desktop\Assignment\df_2022.csv")
df3 = pd.read_csv(r"C:\Users\samiksha.Godghate\Desktop\Assignment\df_2023.csv")
df4 = pd.read_csv(r"C:\Users\samiksha.Godghate\Desktop\Assignment\df_2024.csv")
print(df1.columns)
print(df2.columns)
print(df3.columns)
print(df4.columns)

In [ ]:
# Create a list of DataFrames
data_frames = [df1, df2, df3, df4]

# Concatenate the DataFrames
data = pd.concat(data_frames, ignore_index=True)

# Print the columns of the combined DataFrame to verify
print(data.columns)

# Optionally, print the first few rows of the combined DataFrame to verify the concatenation
print(data.head())

In [7]:
# Check for missing values
missing_values = data.isnull().sum()

# Check for duplicates
duplicates = data.duplicated().sum()

# Handle missing data (example: removing rows with missing values)
data = data.dropna()

In [8]:
# Ensure correct data types
data['user_id'] = data['user_id'].astype(str)
data['bill_id'] = data['bill_id'].astype(str)
data['line_item_amount'] = data['line_item_amount'].astype(float)
data['transaction_date'] = pd.to_datetime(data['transaction_date'])
data['description'] = data['description'].astype(str)
data['inventory_category'] = data['inventory_category'].astype(str)
data['color'] = data['color'].astype(str)
data['size'] = data['size'].astype(str)
data['zone_name'] = data['zone_name'].astype(str)
data['store_name'] = data['store_name'].astype(str)
data['year'] = data['year'].astype(int)

# Check for inconsistencies (example: negative values)
inconsistencies = data[data['line_item_amount'] < 0]

**2. Data Summary & Quality Check**

In [11]:
# Compute basic statistics
total_transactions = data['bill_id'].nunique()
unique_customers = data['user_id'].nunique()
number_of_stores = data['store_name'].nunique()
number_of_products = data['description'].nunique()

# Identify potential data quality issues
negative_values = data[data['line_item_amount'] < 0]
incorrect_timestamps = data[data['transaction_date'] > pd.Timestamp.now()]
mismatched_records = data[data['year'] != data['transaction_date'].dt.year]

# Print summary
print(f"Total Transactions: {total_transactions}")
print(f"Unique Customers: {unique_customers}")
print(f"Number of Stores: {number_of_stores}")
print(f"Number of Products: {number_of_products}")


Total Transactions: 1262425
Unique Customers: 546082
Number of Stores: 451
Number of Products: 72234


**3. Basic Sales Trends**

In [12]:
# Calculate total revenue per year, per store, and per product category
revenue_per_year = data.groupby('year')['line_item_amount'].sum()
revenue_per_store = data.groupby('store_name')['line_item_amount'].sum()
revenue_per_category = data.groupby('inventory_category')['line_item_amount'].sum()

# Identify the top-selling store based on revenue
top_selling_store = revenue_per_store.idxmax()

# Print results
print(f"Revenue per Year:\n{revenue_per_year}")
print(f"Revenue per Store:\n{revenue_per_store}")
print(f"Revenue per Category:\n{revenue_per_category}")
print(f"Top Selling Store: {top_selling_store}")


Revenue per Year:
year
2021    9.209819e+09
2022    1.052845e+10
2023    1.084503e+10
2024    1.072138e+10
Name: line_item_amount, dtype: float64
Revenue per Store:
store_name
Store_1       16172784.0
Store_10      49988932.0
Store_100     60980656.0
Store_101     54543511.0
Store_102     10389584.0
                ...     
Store_95      65274603.0
Store_96      87482067.0
Store_97     228959094.0
Store_98     245644694.0
Store_99     212033152.0
Name: line_item_amount, Length: 451, dtype: float64
Revenue per Category:
inventory_category
Accessories and Innerwear              6.639641e+09
Casual Shirts                          1.560084e+09
Casual Trousers, Shorts and Joggers    3.041851e+09
Denim                                  1.475253e+09
Disregard                              4.961575e+07
Footwear                               8.339311e+08
Formal Shirts                          1.173314e+10
Formal Trousers                        5.936221e+09
Jackets                                2

## Objective 2 – Exploratory Data Analysis & Business Insights

**1. Top-Performing Products & Categories**

In [13]:
# Identify the best-selling products and categories based on revenue and purchase frequency
best_selling_products = data.groupby('description')['line_item_amount'].sum().sort_values(ascending=False).head(10)
best_selling_categories = data.groupby('inventory_category')['line_item_amount'].sum().sort_values(ascending=False).head(10)

# Find the least-performing categories
least_performing_categories = data.groupby('inventory_category')['line_item_amount'].sum().sort_values(ascending=True).head(10)

# Print results
print(f"Best Selling Products:\n{best_selling_products}")
print(f"Best Selling Categories:\n{best_selling_categories}")
print(f"Least Performing Categories:\n{least_performing_categories}")


Best Selling Products:
description
Trolly Bag Promo:DARK BROWN/10    146310135.0
PROMO TRAVEL KIT#BROWN/10          89313255.0
BP-SPECTRUM # WHITE/42             85854456.0
RAYN MIX COLOR # MIX COLOR/43      84096552.0
ROBERT # WHITE/43                  77669887.0
BP-SPECTRUM # WHITE/39             75796801.0
BP-SPECTRUM # WHITE/40             72044324.0
New Teddy # RED/12                 71880511.0
ROY # WHITE/43                     63173577.0
Trolly Bag Promo:black/10          62488051.0
Name: line_item_amount, dtype: float64
Best Selling Categories:
inventory_category
Formal Shirts                          1.173314e+10
Accessories and Innerwear              6.639641e+09
Formal Trousers                        5.936221e+09
Suits                                  3.669663e+09
Casual Trousers, Shorts and Joggers    3.041851e+09
Jackets                                2.422308e+09
T-Shirts                               2.143245e+09
Casual Shirts                          1.560084e+09
Denim 

**2. Seasonality & Demand Trends**

In [14]:
# Identify monthly or yearly sales patterns
data['month'] = data['transaction_date'].dt.month
monthly_sales = data.groupby('month')['line_item_amount'].sum()

# Detect peak shopping periods
peak_month = monthly_sales.idxmax()

# Print results
print(f"Monthly Sales:\n{monthly_sales}")
print(f"Peak Shopping Month: {peak_month}")


Monthly Sales:
month
1     3.911686e+09
2     3.337003e+09
3     2.869986e+09
4     3.647591e+09
5     2.400541e+09
6     2.767673e+09
7     2.741685e+09
8     2.483317e+09
9     2.270645e+09
10    4.537352e+09
11    6.125653e+09
12    4.211546e+09
Name: line_item_amount, dtype: float64
Peak Shopping Month: 11


**3. Customer Purchase Behavior**

In [15]:
# Analyze repeat purchases and customer retention rates
repeat_purchases = data.groupby('user_id')['bill_id'].nunique()
customer_retention = (repeat_purchases > 1).mean()

# Calculate the average transaction value per customer
average_transaction_value = data.groupby('user_id')['line_item_amount'].sum().mean()

# Identify high-value customers (top 10% based on spending)
high_value_customers = data.groupby('user_id')['line_item_amount'].sum().sort_values(ascending=False).head(int(0.1 * unique_customers))

# Print results
print(f"Customer Retention Rate: {customer_retention}")
print(f"Average Transaction Value per Customer: {average_transaction_value}")
print(f"High Value Customers:\n{high_value_customers}")


Customer Retention Rate: 0.4307576517812343
Average Transaction Value per Customer: 75638.23230485531
High Value Customers:
user_id
0822007f39    3.079317e+09
3f19267b63    2.047936e+07
9782189115    1.433528e+07
a5e7a68eaf    1.144159e+07
0e55c85966    1.099358e+07
                  ...     
e4cc9f54a6    1.618200e+05
38ea691d96    1.618200e+05
cd01296b98    1.618200e+05
dd0a23f88c    1.618200e+05
5a99f2e1f5    1.618200e+05
Name: line_item_amount, Length: 54608, dtype: float64


**4. Preliminary Business Recommendations**

Preliminary Business Recommendations : 

1. Boost Sales of Popular Products
- Promote top-selling items like "Trolly Bag Promo" and "BP-SPECTRUM" through discounts and bundle deals.  
- Advertise these products more to attract more buyers.

2. Stock More of Best-Selling Categories
- Ensure Formal Shirts and Accessories & Innerwear are always available since they generate the most revenue.  
- Adjust prices based on demand and seasons to maximize profits.

3. Improve Low-Selling Categories
- Identify reasons why Waistcoats and Zipper Jackets have low sales—price, design, or quality.  
- Introduce new styles, collaborate with influencers, or offer discounts to boost interest.

4. Engage with Customers More
- Provide special discounts to loyal customers to encourage repeat purchases.  
- Use emails and social media to promote new arrivals and limited-time offers.


## Objective 3 – Advanced Business Strategy & Prediction

**1. Customer Segmentation**

In [20]:
from sklearn.cluster import KMeans

# Identify distinct customer groups based on their purchase behavior
customer_data = data.groupby('user_id').agg({
    'line_item_amount': 'sum',
    'bill_id': 'nunique',
    'transaction_date': 'count'
}).reset_index()

# Use KMeans for customer segmentation
kmeans = KMeans(n_clusters=3, random_state=0).fit(customer_data[['line_item_amount', 'bill_id', 'transaction_date']])
customer_data['segment'] = kmeans.labels_

# Calculate the mean for numeric columns only
segment_means = customer_data.groupby('segment')[['line_item_amount', 'bill_id', 'transaction_date']].mean()

# Print segmentation results
print(f"Customer Segmentation:\n{segment_means}")


Customer Segmentation:
         line_item_amount        bill_id  transaction_date
segment                                                   
0            4.939243e+04       1.862162          4.757375
1            3.079317e+09  116762.000000     299532.000000
2            4.600893e+05       6.561861         26.530803


In [23]:
print(customer_data.head())

      user_id  line_item_amount  bill_id  transaction_date  segment
0  000026db73           10780.0        1                 2        0
1  00002dce10            2795.0        1                 1        0
2  00003dc788            9490.0        1                 1        0
3  00004e71b9            9184.0        1                 1        0
4  00005db42b           35425.0        1                 5        0


In [24]:
# Calculate RFM metrics
rfm = data.groupby('user_id').agg({
    'transaction_date': lambda x: (pd.to_datetime('today') - x.max()).days,  # Recency
    'bill_id': 'nunique',  # Frequency
    'line_item_amount': 'sum'  # Monetary Value
}).reset_index()

rfm.columns = ['user_id', 'recency', 'frequency', 'monetary']

# K-means clustering to segment customers
kmeans = KMeans(n_clusters=3, random_state=0).fit(rfm[['recency', 'frequency', 'monetary']])
rfm['segment'] = kmeans.labels_

# Analyze segments
segment_analysis = rfm.groupby('segment').agg({
    'recency': 'mean',
    'frequency': 'mean',
    'monetary': 'mean'
}).reset_index()

print(segment_analysis)


   segment     recency      frequency      monetary
0        0  649.612186       1.862162  4.939243e+04
1        1   38.000000  116762.000000  3.079317e+09
2        2  430.583504       6.561861  4.600893e+05


Customer Segmentation Analysis

Segment 0:
- Persistency ChangeRecency:Persistency Change 649.61 days
- Persistency ChangeFrequency:Persistency Change 1.86 transactions
- Persistency ChangeMonetary:Persistency Change ₹49,392.43
- Persistency ChangeCharacteristics:Persistency Change These customers have not made a purchase in a long time, make very few transactions, and spend a relatively low amount.
- Persistency ChangeEngagement Strategy:Persistency Change
  - Persistency ChangeRe-engagement Campaign:Persistency Change Target these customers with re-engagement emails or SMS, offering exclusive discounts or promotions to encourage them to return.
  - Persistency ChangePersonalized Recommendations:Persistency Change Use their past purchase data to recommend products they might be interested in.
  - Persistency ChangeLoyalty Points:Persistency Change Offer loyalty points for their next purchase to incentivize a return visit.

Segment 1:
- Persistency ChangeRecency:Persistency Change 38.00 days
- Persistency ChangeFrequency:Persistency Change 116,762.00 transactions
- Persistency ChangeMonetary:Persistency Change ₹3,079,317,000
- Persistency ChangeCharacteristics:Persistency Change These are high-value customers who make frequent purchases and spend a significant amount.
- Persistency ChangeEngagement Strategy:Persistency Change
  - Persistency ChangeVIP Treatment:Persistency Change Provide exclusive perks, early access to new products, and personalized customer service.
  - Persistency ChangeTiered Loyalty Program:Persistency Change Implement a tiered loyalty system where these customers receive the highest tier benefits, such as larger discounts, free shipping, and special events.
  - Persistency ChangeFeedback Loop:Persistency Change Regularly seek their feedback to understand their needs better and tailor offerings accordingly.

Segment 2:
- Persistency ChangeRecency:Persistency Change 430.58 days
- Persistency ChangeFrequency:Persistency Change 6.56 transactions
- Persistency ChangeMonetary:Persistency Change ₹460,089.30
- Persistency ChangeCharacteristics:Persistency Change These customers make more frequent purchases than Segment 0 but still have a relatively long recency and moderate spending.
- Persistency ChangeEngagement Strategy:Persistency Change
  - Persistency ChangeRegular Engagement:Persistency Change Send regular newsletters and updates about new products, sales, and promotions.
  - Persistency ChangeSeasonal Offers:Persistency Change Target them with seasonal offers and discounts to encourage more frequent purchases.
  - Persistency ChangeLoyalty Rewards:Persistency Change Offer loyalty rewards for repeat purchases to incentivize continued engagement.



**2. Loyalty Program Strategy**

    Data-Driven Approach to Increase Customer Retention:-
-  Segment Analysis:  Use the segmentation data to tailor loyalty programs specifically for each segment.
-  Personalized Offers:  Leverage customer purchase history to offer personalized discounts and recommendations.
-  Engagement Metrics:  Track engagement metrics such as open rates, click-through rates, and conversion rates to refine the loyalty program continuously.


    Tiered Loyalty System Benefits:- 
-  Discounts for High Spenders:  Offer increasing discounts or cashback for customers who spend more.
-  Exclusive Perks:  Provide exclusive perks like early access to sales, special events, and personalized shopping experiences for high-tier customers.
-  Points System:  Implement a points system where customers earn points for every purchase, which can be redeemed for discounts or free products.
-  Engagement Rewards:  Reward customers for engaging with the brand, such as writing reviews, referring friends, or participating in surveys.

**3. Predictive Modeling**

In [22]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Encode categorical target variable
label_encoder = LabelEncoder()
data['inventory_category_encoded'] = label_encoder.fit_transform(data['inventory_category'])

# Aggregate past purchase trends
customer_purchase_history = data.groupby('user_id').agg({
    'inventory_category_encoded': lambda x: x.value_counts().index[0]
}).reset_index()

# Split data into training and testing sets
X = customer_purchase_history.drop(columns=['inventory_category_encoded'])
X['user_id'] = X['user_id'].astype('category').cat.codes

y = customer_purchase_history['inventory_category_encoded']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


# Train a predictive model
model = RandomForestClassifier(random_state=0)
model.fit(X_train, y_train)

# Predict the next likely purchase
predictions = model.predict(X_test)

# Print predictive modeling results
print(f"Predictive Modeling Accuracy: {model.score(X_test, y_test)}")


Predictive Modeling Accuracy: 0.16404955272530833


In [25]:

prediction_data = data[['user_id']].drop_duplicates()
prediction_data['user_id'] = prediction_data['user_id'].astype('category').cat.codes

# Make predictions
predictions = model.predict(prediction_data)

# Convert the encoded predictions back to the original categories
predicted_categories = label_encoder.inverse_transform(predictions)

# Create a DataFrame to store the results
prediction_results = pd.DataFrame({
    'user_id': data['user_id'].drop_duplicates(),
    'predicted_next_purchase': predicted_categories
})

# Print the prediction results
print(prediction_results)


            user_id    predicted_next_purchase
0        12668fb1ff              Formal Shirts
11       059e07d4ed              Formal Shirts
18       65f8d6a6c3  Accessories and Innerwear
29       4297fe4bad            Formal Trousers
30       8947a94292              Formal Shirts
...             ...                        ...
3499985  04b32f18a3                      Suits
3499988  7b2a72f742                      Suits
3499991  8f7ed3765a                    Jackets
3499995  034800c0c7                  Waistcoat
3499997  a2c78d3acb              Formal Shirts

[546082 rows x 2 columns]


Improving Predictive Model Accuracy

An accuracy of 16.4% is relatively low. Here are some steps to improve the model's accuracy:

  1. Feature Engineering
-  Add More Features : Include additional relevant features.
-  Feature Interactions : Create interaction terms between features.

  2. Handling Imbalanced Data
-  Resampling : Oversample the minority class or undersample the majority class.
-  Class Weights : Adjust class weights in the model.

  3. Model Selection
-  Try Different Algorithms : Experiment with Gradient Boosting, XGBoost, LightGBM, or neural networks.
-  Hyperparameter Tuning : Use Grid Search or Random Search.

  4. Ensemble Methods
-  Stacking : Combine predictions from multiple models.
-  Bagging and Boosting : Use techniques like Random Forest or Gradient Boosting.

  5. Cross-Validation
-  K-Fold Cross-Validation : Ensure consistent performance across different data subsets.

**4. Innovative Business Solutions**

One unique insight that could significantly impact business growth is identifying and targeting high-value customers who are likely to make repeat purchases. By focusing on these customers, the business can increase customer lifetime value (CLV) and drive sustainable growth.

1.  Customer Segmentation:
    - Segment customers based on their purchase history, frequency, and value.
    - Identify high-value segments and tailor marketing strategies to retain and upsell to these customers.

2.  Dynamic Pricing:
    - Implement dynamic pricing strategies based on customer segments and predicted purchase behavior.
    - Offer personalized discounts to high-value customers to encourage repeat purchases.

3.  Inventory Optimization:
    - Use predictive insights to optimize inventory levels for popular products.
    - Ensure that high-demand products are always in stock to avoid lost sales.
